# AOG Text-to-Image Example

This notebook demonstrates how to use AOG Text-to-Image API to generate images from text descriptions.

## Table of Contents
1. [Environment Setup](#1-Environment Setup)
2. [Helper Functions](#2-Helper Functions)
3. [Basic Image Generation](#3-Basic Image Generation)
4. [Different Style Prompts](#4-Different Style Prompts)
5. [Save Images](#5-Save Images)
6. [Batch Generation](#6-Batch Generation)

## 1. Environment Setup

First, import necessary libraries and configure AOG service connection parameters.

In [ ]:
# Import necessary libraries
import requests
import base64
from io import BytesIO
from PIL import Image
import json
from typing import Optional, Dict, List
from datetime import datetime

# AOG service configuration
AOG_BASE_URL = "http://localhost:16688"  # Modify this if service is at a different address
AOG_API_SPEC_VERSION = "v0.2"
DEFAULT_MODEL_IMAGE = "OpenVINO/LCM_Dreamshaper_v7-fp16-ov"  # Default image generation model

print("✅ Configuration complete")
print(f"📍 AOG service address: {AOG_BASE_URL}")
print(f"🎨 Default model: {DEFAULT_MODEL_IMAGE}")

## 2. Helper Functions

定义Call AOG Text-to-Image API 和处理图像Helper Functions.

In [ ]:
def call_aog_text_to_image(
    prompt: str, 
    model: str = DEFAULT_MODEL_IMAGE
) -> Optional[Dict]:
    """
    Call AOG Text-to-Image API
    
    Args:
        prompt: Text description for generating images
        model: Model name to use
    
    Returns:
        API 响应 JSON 数据,如果失败则Returns None
    """
    try:
        # 构造 API 端点 URL
        url = f"{AOG_BASE_URL}/aog/{AOG_API_SPEC_VERSION}/services/text-to-image"
        
        # 构造请求数据
        payload = {
            "model": model,
            "prompt": prompt
        }
        
        print(f"📤 Sending request到: {url}")
        print(f"📝 Tip词: {prompt}")
        
        # 发送 POST 请求
        response = requests.post(url, json=payload, timeout=60)
        
        # 检查响应状态
        response.raise_for_status()
        
        print("✅ 图像Generation successful")
        
        # Returns JSON 响应
        return response.json()
        
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to AOG service, please ensure service is running")
        print(f"   Check service address: {AOG_BASE_URL}")
        return None
        
    except requests.exceptions.Timeout:
        print("❌ Request timeout,图像生成可能需要较长时间")
        print("   建议: 增加超时时间或简化Tip词")
        return None
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ API call failed: {e}")
        print(f"   Response content: {response.text}")
        return None
        
    except Exception as e:
        print(f"❌ Unknown error: {e}")
        return None


def decode_and_display_image(base64_string: str) -> Optional[Image.Image]:
    """
    解码并显示 base64 编码图像
    
    Args:
        base64_string: base64 编码图像字符串
    
    Returns:
        PIL Image 对象,如果失败则Returns None
    """
    try:
        # 解码 base64 字符串
        image_data = base64.b64decode(base64_string)
        
        # 使用 PIL 打开图像
        image = Image.open(BytesIO(image_data))
        
        # 在 notebook 中显示图像
        display(image)
        
        # 打印图像信息
        print(f"\n📊 图像信息: {image.size[0]}x{image.size[1]} 像素, 格式: {image.format}")
        
        return image
        
    except Exception as e:
        print(f"❌ 图像解码失败: {e}")
        return None


def generate_and_display(prompt: str, model: str = DEFAULT_MODEL_IMAGE) -> Optional[Image.Image]:
    """
    生成并显示图像便捷函数
    
    Args:
        prompt: 文本描述
        model: Model name
    
    Returns:
        PIL Image 对象,如果失败则Returns None
    """
    print("\n" + "="*60)
    
    # 调用 API 生成图像
    response = call_aog_text_to_image(prompt, model)
    
    if response is None:
        print("="*60 + "\n")
        return None
    
    # 提取 base64 图像数据
    if "data" in response and len(response["data"]) > 0:
        base64_image = response["data"][0].get("b64_json", "")
        
        if base64_image:
            print("\n🖼️  生成图像:")
            image = decode_and_display_image(base64_image)
            print("="*60 + "\n")
            return image
        else:
            print("⚠️  响应中没有图像数据")
    else:
        print("⚠️  Abnormal response format")
        print(json.dumps(response, indent=2, ensure_ascii=False))
    
    print("="*60 + "\n")
    return None

print("✅ Helper Functions定义完成")

## 3. Basic Image Generation

最简单图像生成示例:提供一个文本描述,生成对应图像.

In [ ]:
# 简单Tip词
prompt = "A beautiful sunset over mountains"

image = generate_and_display(prompt)

## 4. Different Style Prompts

通过添加风格关键词,可以生成不同艺术风格图像.

### 4.1 照片级真实风格

In [ ]:
# 照片级真实风格
prompt = "A cute cat sitting on a windowsill, natural lighting, photorealistic, highly detailed"

image = generate_and_display(prompt)

### 4.2 数字艺术风格

In [ ]:
# 数字艺术风格
prompt = "A magical forest with glowing mushrooms, fantasy landscape, digital art, vibrant colors"

image = generate_and_display(prompt)

### 4.3 油画风格

In [ ]:
# 油画风格
prompt = "A vase of sunflowers on a table, oil painting style, impressionist, warm colors"

image = generate_and_display(prompt)

### 4.4 科幻风格

In [ ]:
# 科幻风格
prompt = "Futuristic city skyline at night, neon lights, cyberpunk style, highly detailed, 8k"

image = generate_and_display(prompt)

### 4.5 水彩画风格

In [ ]:
# 水彩画风格
prompt = "A peaceful lake with mountains in the background, watercolor painting, soft colors, dreamy"

image = generate_and_display(prompt)

## 5. Save Images

演示如何将生成图像保存到文件.

In [ ]:
def save_image(image: Image.Image, filename: str = None) -> str:
    """
    Save Images到文件
    
    Args:
        image: PIL Image 对象
        filename: 文件名(可选,默认使用时间戳)
    
    Returns:
        保存文件路径
    """
    if image is None:
        print("⚠️  没有图像可保存")
        return None
    
    try:
        # 如果没有指定文件名,使用时间戳
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"generated_image_{timestamp}.png"
        
        # Save Images
        image.save(filename)
        
        print(f"✅ 图像已保存到: {filename}")
        return filename
        
    except Exception as e:
        print(f"❌ Save Images失败: {e}")
        return None

# 生成并Save Images
prompt = "A serene Japanese garden with cherry blossoms, peaceful atmosphere, beautiful lighting"
image = generate_and_display(prompt)

if image:
    save_image(image, "japanese_garden.png")

## 6. Batch Generation

演示如何Batch Generation多个图像.

In [ ]:
# 定义多个Tip词
prompts = [
    "A red apple on a wooden table, natural lighting",
    "A blue butterfly on a flower, macro photography",
    "A golden retriever puppy playing in grass, sunny day",
    "A steaming cup of coffee with latte art, cozy cafe"
]

# Batch Generation图像
generated_images = []

for i, prompt in enumerate(prompts, 1):
    print(f"\n🎨 生成第 {i}/{len(prompts)} 张图像...")
    image = generate_and_display(prompt)
    
    if image:
        generated_images.append(image)
        # 可选:保存每张图像
        # save_image(image, f"batch_image_{i}.png")

print(f"\n✅ 成功生成 {len(generated_images)}/{len(prompts)} 张图像")

### 6.1 创建图像网格显示

In [ ]:
import matplotlib.pyplot as plt

def display_image_grid(images: List[Image.Image], cols: int = 2):
    """
    以网格形式显示多张图像
    
    Args:
        images: PIL Image 对象列表
        cols: 每行显示图像数量
    """
    if not images:
        print("⚠️  没有图像可显示")
        return
    
    # 计算行数
    rows = (len(images) + cols - 1) // cols
    
    # 创建图形
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    
    # 如果只有一行或一列,确保 axes 是二维数组
    if rows == 1:
        axes = [axes]
    if cols == 1:
        axes = [[ax] for ax in axes]
    
    # 显示图像
    for i, image in enumerate(images):
        row = i // cols
        col = i % cols
        axes[row][col].imshow(image)
        axes[row][col].axis('off')
        axes[row][col].set_title(f"Image {i+1}")
    
    # 隐藏多余子图
    for i in range(len(images), rows * cols):
        row = i // cols
        col = i % cols
        axes[row][col].axis('off')
    
    plt.tight_layout()
    plt.show()

# 显示生成图像网格
if generated_images:
    print("\n🖼️  图像网格显示:")
    display_image_grid(generated_images, cols=2)

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ Configure AOG service connection
2. ✅ 调用 Text-to-Image API 生成图像
3. ✅ 编写有效Tip词
4. ✅ 处理 base64 编码图像数据
5. ✅ 在 Jupyter Notebook 中显示图像
6. ✅ 保存生成图像到文件
7. ✅ Batch Generation多张图像
8. ✅ 创建图像网格显示
9. ✅ Handle API errors and exceptions

### Tip词编写技巧回顾

一个好Tip词通常包含:
- 📝 **主体描述**: 清晰描述你想要什么
- 🎨 **风格关键词**: digital art, oil painting, photorealistic 等
- ✨ **质量词**: highly detailed, 8k, professional 等
- 💡 **光照和氛围**: golden hour, dramatic lighting, moody 等

### Next Steps

- 尝试更多创意Tip词
- 实验不同艺术风格
- Check out [文本生成示例](../text-generation/text-generation-demo.ipynb)
- Read [AOG API 文档](../../docs/) for more features
- 探索Tip词工程最佳实践

**祝你创作愉快!** 🎨✨